<a href="https://colab.research.google.com/github/FadelYang/craft-it-object-detection-model/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#importing required Libraries
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from google.colab import files

# Preparing Dataset

## Mount to Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd drive/MyDrive/'Colab Notebooks'

/content/drive/MyDrive/Colab Notebooks


## Load Dataset



In [24]:
!rm -r dataset
!rm data
# !wget https://app.roboflow.com/ds/gzGSPSpsan?key=CrzyeSBgrQ
# !mv gzGSPSpsan?key=CrzyeSBgrQ data
# !mkdir dataset
# !unzip data -d dataset

In [5]:
## Load label dataset.
dataset_dir = os.path.join(os.getcwd(), 'dataset')
train_label_df = pd.read_csv(os.path.join(dataset_dir, 'train', '_classes.csv'))
validation_label_df = pd.read_csv(os.path.join(dataset_dir, 'valid', '_classes.csv'))
train_images_dir = os.path.join(dataset_dir, 'train')
validation_images_dir = os.path.join(dataset_dir, 'valid')

print(train_images_dir)

/content/drive/MyDrive/Colab Notebooks/dataset/train


In [6]:
print(train_label_df.head())
print(validation_label_df.head())

                                           filename   aluminiumfoil   bottle  \
0  1196_jpg.rf.68689b8efc68113f5ffa321d8c02fe0b.jpg               0        0   
1   757_jpg.rf.67b5d7ce20e152ea0f2b8783ffe65301.jpg               0        0   
2   344_jpg.rf.6725ba8501f47157d433edc3e1420a85.jpg               0        1   
3    65_jpg.rf.65d079f8e8e3840c836b3902febc65c8.jpg               0        0   
4   414_jpg.rf.665baa402c9f84c0416955ebe0ad3925.jpg               0        0   

    bottlecap   can   carton   cup   paper   plastic   straw  
0           0     0        0     0       0         0       1  
1           0     0        0     1       0         0       0  
2           0     1        0     0       0         0       0  
3           1     0        0     0       0         0       0  
4           0     1        0     0       0         0       0  
                                          filename   aluminiumfoil   bottle  \
0  121_jpg.rf.030d2a2ac6f40bab2e9bea71c50b7716.jpg            

In [7]:
# !rm -r train

In [8]:
train_label_df.columns = train_label_df.columns.str.strip()
validation_label_df.columns = validation_label_df.columns.str.strip()

In [9]:
# # Check the data.
# import os

# train_image_filenames = train_label_df['filename'].tolist()
# validation_image_filenames = validation_label_df['filename'].tolist()

# train_image_paths = [os.path.join(train_images_dir, fname) for fname in train_image_filenames]
# validation_image_paths = [os.path.join(validation_images_dir, fname) for fname in validation_image_filenames]

# # Check if all files exist
# missing_train_images = [img for img in train_image_paths if not os.path.exists(img)]
# missing_validation_images = [img for img in validation_image_paths if not os.path.exists(img)]

# print(f"Missing train images: {missing_train_images}")
# print(f"Missing validation images: {missing_validation_images}")

In [20]:
columns = [
    "aluminiumfoil",
    "bottle",
    "bottlecap",
    "can",
    "carton",
    "cup",
    "paper",
    "plastic",
    "straw"
]

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.)
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.)


if os.path.exists(train_images_dir):
  train_generator = train_datagen.flow_from_dataframe(
      dataframe=train_label_df,
      directory=train_images_dir,
      # target_size=(224, 224),
      x_col='filename',
      y_col=columns,
      batch_size=32,
      seed=42,
      class_mode='raw',
      shuffle=True,
)

if os.path.exists(validation_images_dir):
  validation_generator = validation_datagen.flow_from_dataframe(
      dataframe=validation_label_df,
      directory=validation_images_dir,
      # target_size=(224, 224),
      x_col='filename',
      y_col=columns,
      batch_size=32,
      class_mode='raw',
      seed=42,
      shuffle=False,
)

Found 2331 validated image filenames.
Found 227 validated image filenames.


# Create Model and Training

## Build Model



In [11]:
resnet_model = tf.keras.applications.ResNet50(
    include_top=False,
    weights='imagenet',
    input_tensor=tf.keras.Input(shape=(640, 480, 3))
)

In [12]:
# Freeze layers in ResNet50
resnet_model.trainable = False

print(resnet_model)
print(train_generator)

In [13]:
x = resnet_model.output.shape
print(x)

(None, 7, 7, 2048)


In [21]:
inputs = tf.keras.layers.Input(shape=(640, 480, 3))
x = resnet_model(inputs)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
# x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
# x = tf.keras.layers.MaxPooling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.MaxPooling2D((2, 2))(x)

x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(9, activation='sigmoid')(x)

# Create the model using the functional API
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [22]:
model.compile(
    tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='binary_crossentropy',
    metrics=[tf.keras.metrics.AUC(name='auc'), 'accuracy']
)

In [23]:
history = model.fit(
    train_generator,
    epochs=5,
    validation_data=validation_generator,
)

Epoch 1/5
73/73 [==============================] - 33s 345ms/step - loss: 0.3880 - auc: 0.6088 - accuracy: 0.1999 - val_loss: 0.3608 - val_auc: 0.5360 - val_accuracy: 0.1145
Epoch 2/5
73/73 [==============================] - 21s 292ms/step - loss: 0.3691 - auc: 0.6297 - accuracy: 0.1982 - val_loss: 0.3597 - val_auc: 0.5336 - val_accuracy: 0.1145
Epoch 3/5
73/73 [==============================] - 21s 287ms/step - loss: 0.3656 - auc: 0.6364 - accuracy: 0.2081 - val_loss: 0.3665 - val_auc: 0.5377 - val_accuracy: 0.1145
Epoch 4/5
73/73 [==============================] - 21s 281ms/step - loss: 0.3629 - auc: 0.6470 - accuracy: 0.2244 - val_loss: 0.3709 - val_auc: 0.5349 - val_accuracy: 0.1278
Epoch 5/5
73/73 [==============================] - 20s 275ms/step - loss: 0.3618 - auc: 0.6513 - accuracy: 0.2196 - val_loss: 0.3593 - val_auc: 0.5308 - val_accuracy: 0.1101
